## Checking the heudiconv result to see if all the dicoms are correctly coverted to nifti

  * Project: MINI
  * Number of sessions: 97
  * Number of subjects: 66
  * T1w: 1 MPRAGE
  * Func: 2 runs, block design
  * 287 volumns per run (For S061-T02-run01 is 188)

### 1. Using PyBIDS for query for all the niftis to see if we can get them all

In [42]:
import bids
from bids import BIDSLayout
import pandas as pd
import os
import os.path as Path
import nibabel as nib
#%%

basedir='/bcbl/home/public/Gari/MINI'
bidsdir=Path.join(basedir,'BIDS_BLQfunc_T1')

layout= BIDSLayout(bidsdir)

In [ ]:
t1= layout.get(suffix="T1w", extension='nii.gz')
len(t1)

In [61]:
all_func= layout.get(datatype='func',extension='nii.gz', return_type='file')
len(all_func)

192

In [62]:
all_func


['/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S001/ses-T01/func/sub-S001_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S001/ses-T01/func/sub-S001_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S002/ses-T01/func/sub-S002_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S002/ses-T01/func/sub-S002_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S003/ses-T01/func/sub-S003_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S003/ses-T01/func/sub-S003_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S004/ses-T01/func/sub-S004_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S004/ses-T01/func/sub-S004_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BI

It should be **194** but here we only get *192* which is not good, so I am going to get 

97

In [6]:
subjects = layout.get(target='subject', return_type='id')

In [7]:
subjects

['S001',
 'S002',
 'S003',
 'S004',
 'S005',
 'S006',
 'S007',
 'S008',
 'S009',
 'S010',
 'S011',
 'S012',
 'S013',
 'S014',
 'S015',
 'S016',
 'S017',
 'S018',
 'S019',
 'S020',
 'S021',
 'S022',
 'S023',
 'S024',
 'S025',
 'S026',
 'S027',
 'S028',
 'S029',
 'S030',
 'S031',
 'S032',
 'S033',
 'S034',
 'S035',
 'S041',
 'S042',
 'S043',
 'S044',
 'S045',
 'S046',
 'S047',
 'S048',
 'S049',
 'S050',
 'S051',
 'S052',
 'S053',
 'S054',
 'S055',
 'S056',
 'S057',
 'S058',
 'S059',
 'S060',
 'S061',
 'S062',
 'S063',
 'S064',
 'S065',
 'S066',
 'S067',
 'S068',
 'S069',
 'S070',
 'S071']

In [8]:
count_ses=0
count_func=0
for sub in subjects:
    sess=layout.get(subject=sub, target='session', return_type='id')
    for ses in sess:
        func= layout.get(subject=sub, session=ses, datatype='func', extension='nii.gz', return_type='file')
        #print(f"there are {len(func)} func files for subject: {sub}, session: {ses}") 
        if len(func) !=2:
            print(f"sub-{sub}-ses{ses} have strange func runs: which is {len(func)}")
        count_ses+=1
        count_func+=len(func)

sub-S007-sesT01 have strange func runs: which is 1
sub-S069-sesT01 have strange func runs: which is 1


In [14]:
count_func

192

In [15]:
count_ses

97

Now, going to check line by line in the mini db txt file, that if the dicoms number is the same as the func

In [13]:

mapping_table=pd.read_csv(Path.join(basedir,'dicoms_BLQfunc_T1','ID_mapping.tsv'), sep='\t', header=0)

mapping=mapping_table.loc[0:96,['OsirixID','BIDS_sub', 'BIDS_ses','MINID']]

dicom_folder=Path.join(basedir,'DATA','dicoms')

In [22]:
mapping

,OsirixID,BIDS_sub,BIDS_ses,MINID
0,S001_MINIONCE_4126,S001,T01,S001
1,S002_MINIONCE_4481,S002,T01,S002
2,S003_MINIONCE_4343,S003,T01,S003
3,S004_MINIONCE_4391,S004,T01,S004
4,S005_MINIONCE_2138,S005,T01,S005
...,...,...,...,...
92,S028_DAY2_MINITWICE_4092,S068,T02,S098
93,S029_DAY2_MINITWICE_5186,S069,T02,S099
94,S030_DAY2_MINITWICE_1820,S070,T02,S100
95,S031_DAY1_MINITWICE_5547,S071,T01,S101


In [28]:
for i,row in minidb.iterrows():
    bid_sub=minidb.iloc[i,0]
    bid_ses=minidb.iloc[i,1]

    mini_dicom=mapping.loc[(mapping['BIDS_ses']==bid_ses) & (mapping['BIDS_sub']==bid_sub),'OsirixID'].values[0]

In [31]:
mini_dicom

'S007_MINIONCE_4838'

In [34]:
osid_name=mapping.loc[(mapping['BIDS_ses']=='T01') & (mapping['BIDS_sub']=='S007'),'OsirixID'].values[0]
mini_name=mapping.loc[(mapping['BIDS_ses']=='T01') & (mapping['BIDS_sub']=='S007'),'MINID'].values[0]

In [35]:
mini_name

'S007'

In [ ]:
count_ses=0
count_func=0
for sub in subjects:
    sess=layout.get(subject=sub, target='session', return_type='id')
    for ses in sess:
        func= layout.get(subject=sub, session=ses, datatype='func', extension='nii.gz', return_type='file')
        #print(f"there are {len(func)} func files for subject: {sub}, session: {ses}") 
        if len(func) !=2:
            print(f"sub-{sub}-ses{ses} have strange func runs: which is {len(func)}")
        count_ses+=1
        count_func+=len(func)



In [39]:
func

['/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S001/ses-T01/func/sub-S001_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S001/ses-T01/func/sub-S001_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S002/ses-T01/func/sub-S002_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S002/ses-T01/func/sub-S002_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S003/ses-T01/func/sub-S003_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S003/ses-T01/func/sub-S003_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S004/ses-T01/func/sub-S004_ses-T01_task-MINIblock_run-01_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BIDS_BLQfunc_T1/sub-S004/ses-T01/func/sub-S004_ses-T01_task-MINIblock_run-02_bold.nii.gz',
 '/bcbl/home/public/Gari/MINI/BI

In [58]:
for i in func:
    img=nib.load(i)
    func_name= i.split('/')[-1].split('_')
    sub=func_name[0]
    ses=func_name[1]
    run=func_name[3]
    #print(f'For {sub}, {ses}, {run}, the fMRI has {img.shape[3]} volumn')
    if img.shape[3] != 287:
        print(f'For {sub}, {ses}, {run}, the fMRI has {img.shape[3]} volumn')
    osid_name=mapping.loc[(mapping['BIDS_ses']=='T02') & (mapping['BIDS_sub']=='S061'),'OsirixID'].values[0]
    mini_name=mapping.loc[(mapping['BIDS_ses']=='T02') & (mapping['BIDS_sub']=='S061'),'MINID'].values[0]

For sub-S061, ses-T02, run-01, the fMRI has 188 volumn


In [59]:
osid_name

'S021_DAY2_MINITWICE_3732'

In [60]:
mini_name

'S091'